In [3]:
# Install necessary libraries (if not already installed)
!pip install pandas numpy matplotlib seaborn plotly

# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
def load_and_clean_data(filepath):
    df = pd.read_csv(filepath)
    
    # Dropping unnecessary columns
    columns_to_drop = ["img_link", "product_link", "user_id", "user_name", "review_id", "review_title", "review_content"]
    df.drop(columns=columns_to_drop, errors="ignore", inplace=True)
    
    # Extracting main category
    df["category_top"] = df["category"].astype(str).apply(lambda x: x.split('|')[0])
    
    # Cleaning and conversion to numeric columns
    for col in ["discounted_price", "actual_price", "rating_count"]:
        df[col] = df[col].astype(str).str.replace(r"[^\d.]", "", regex=True)
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    if "discount_percentage" in df.columns:
        df["discount_percentage"] = df["discount_percentage"].astype(str).str.replace("%", "").astype(float)
    
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
    
    # Handling missing values
    df["rating"].fillna(df["rating"].mean(), inplace=True)
    df["rating_count"].fillna(df["rating_count"].median(), inplace=True)
    df.dropna(inplace=True)
    
    # Profit margin column
    df["profit_margin"] = df["actual_price"] - df["discounted_price"]
    
    return df

data = load_and_clean_data("amazon.csv")

data.head()

C:\Users\zaina\AppData\Local\Temp\ipykernel_31896\2245375129.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["rating"].fillna(df["rating"].mean(), inplace=True)
C:\Users\zaina\AppData\Local\Temp\ipykernel_31896\2245375129.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,category_top,profit_margin
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64.0,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,Computers&Accessories,700.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43.0,4.0,43994.0,"Compatible with all Type C enabled devices, be...",Computers&Accessories,150.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90.0,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,Computers&Accessories,1700.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53.0,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,Computers&Accessories,370.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61.0,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,Computers&Accessories,245.0


In [5]:
# Cards to be shown on streamlit
total_products = data.shape[0]
average_rating = round(data["rating"].mean(), 2)
top_category = data["category_top"].mode()[0]
average_rating_count = int(data["rating_count"].mean())

print(f"Total Products: {total_products}")
print(f"Average Rating: {average_rating}")
print(f"Top Category: {top_category}")
print(f"Average Rating Count: {average_rating_count}")


Total Products: 1465
Average Rating: 4.1
Top Category: Electronics
Average Rating Count: 18277


In [6]:
fig1 = px.histogram(data, x="rating", nbins=10, marginal="box",
                   title="Distribution of Product Ratings")
fig1.show()

In [7]:
fig2 = px.scatter(data, x="actual_price", y="discount_percentage", color="category_top", 
                 size="rating_count", title="Price vs Discount Percentage")
fig2.update_layout(margin=dict(l=100, r=100, b=100, t=100), height=485)
fig2.show()

In [8]:
corr_matrix = data[["discounted_price", "actual_price", "rating", "rating_count"]].corr().round(1)
fig = px.imshow(corr_matrix, text_auto=True, title="Correlation Heatmap")
fig.show()

In [9]:
avg_profit_margin = data.groupby("category_top")["profit_margin"].mean().round(0).astype(int).reset_index()

fig = px.bar(
    avg_profit_margin, 
    x="category_top", 
    y="profit_margin", 
    color="profit_margin",
    title="Average Profit Margin by Category",
    labels={"category_top": "Category", "profit_margin": "Profit Margin (₹)"},
    text="profit_margin"
)
fig.show()

In [10]:
top_categories = data["category_top"].value_counts().head(10)

fig = px.bar(
    top_categories,
    x=top_categories.index,
    y=top_categories.values,
    title="Product Distribution by Top-Level Category",
    labels={"y": "Number of Products", "x": "Category"}
)

# Fixes x-label bug
fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Number of Products"
)

fig.show()

In [ ]:
# Donut Chart
fig_donut = px.pie(
    names=top_categories.index,
    values=top_categories.values,
    title="Product Distribution by Category (Donut Chart)",
    labels={"values": "Number of Products", "names": "Category"},
    hole=0.4,
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig_donut.show()